In [16]:
from pymongo import MongoClient
import feedparser
from pprint import pprint
import datetime
import random

def build_heading_quiz(answers):

    # open database connection
    username="root"
    password="mongo"
    client = MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))
    db = client.headings

    # get a random heading
    heading = list(db.headings.aggregate([{ "$sample": { "size": 1 } }]))[0]
    heading_source_name = heading["_source"]["name"]
    #pprint(heading["title"])
    #pprint(heading_source_name)

    # pick additional answers-1 random sources
    sources_all = [x["name"] for x in db.sources.find()]
    sources = [heading_source_name, *random.sample(list(filter(lambda x: x != heading_source_name, sources_all)), answers - 1)]
    random.shuffle(sources)
    #pprint(sources)

    # find index of headings source
    index = sources.index(heading_source_name)
    #pprint(index)
    
    return {"title": heading["title"], "sources": sources, "index": index, "link": heading["link"]}

In [17]:
import logging
import os

from telegram import (Poll, ParseMode, KeyboardButton, KeyboardButtonPollType,
                      ReplyKeyboardMarkup, ReplyKeyboardRemove)
from telegram.ext import (Updater, CommandHandler, PollAnswerHandler, PollHandler, MessageHandler,
                          Filters)

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)


def start(update, context):
    """Inform user about what this bot can do"""
    update.message.reply_text('Please select /quiz, /quiz3 or /quiz4 to get a Quiz')
    
def quiz3(update, context):
    quiz(update, context, 3)

def quiz4(update, context):
    quiz(update, context, 4)

    
def quiz(update, context, qs=2):
    
    q = build_heading_quiz(qs)
    
    """Send a predefined poll"""
    questions = q["sources"]
    message = update.effective_message.reply_poll("Where was this heading published?\n\n" + q["title"],
                                                  questions, type=Poll.QUIZ, correct_option_id=q["index"])
    # Save some info about the poll the bot_data for later use in receive_quiz_answer
    payload = {message.poll.id: {"chat_id": update.effective_chat.id,
                                 "message_id": message.message_id,
                                 "q": q,
                                 "qs": qs}}
    context.bot_data.update(payload)

def post_quiz(context):
    pprint(context)
    """Send the alarm message."""
    quiz_data = context.job.context
    context.bot.send_message(quiz_data["chat_id"], text=quiz_data["q"]["link"] + "\n\n" + "/quiz, /quiz3 or /quiz4")  
            

def receive_quiz_answer(update, context):
    pprint(update)
    pprint(context)
    pprint(update.poll)
    pprint(update.poll_answer)
    pprint(context.bot_data)
    
    """Close quiz after three participants took it"""
    # the bot can receive closed poll updates we don't care about
    if update.poll.is_closed:
        return
    if update.poll.total_voter_count == 3:
        try:
            quiz_data = context.bot_data[update.poll.id]
        # this means this poll answer update is from an old poll, we can't stop it then
        except KeyError:
            return
        context.bot.stop_poll(quiz_data["chat_id"], quiz_data["message_id"])
        
    if update.poll:
        pprint("what's next")
        quiz_data = context.bot_data[update.poll.id]
        chat_id = quiz_data["chat_id"]
        new_job = context.job_queue.run_once(post_quiz, 2, context=quiz_data)

        
def help_handler(update, context):
    """Display a help message"""
    update.message.reply_text("Use /quiz, /quiz3 or /quiz4 to test this "
                              "bot.")


def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater(os.environ['TEL_BOT_TOKEN'], use_context=True)
    dp = updater.dispatcher
    dp.add_handler(CommandHandler('start', start))
    dp.add_handler(CommandHandler('quiz', quiz))
    dp.add_handler(CommandHandler('quiz3', quiz3))
    dp.add_handler(CommandHandler('quiz4', quiz4))
    dp.add_handler(PollHandler(receive_quiz_answer))
    dp.add_handler(CommandHandler('help', help_handler))

    # Start the Bot
    updater.start_polling()

    # Run the bot until the user presses Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT
    updater.idle()

In [ ]:
main()

None
{'6014925720352981094': {'chat_id': 327434945,
                         'message_id': 225,
                         'q': {'index': 1,
                               'link': 'https://www.independent.co.uk/news/world/americas/us-politics/amy-coney-barrett-past-decisions-rulings-beliefs-supreme-court-trump-rbg-b556203.html',
                               'sources': ['The Sun', 'The Independent'],
                               'title': 'Amy Coney Barrett: The most '
                                        "controversial decisions by Trump's "
                                        'Supreme Court frontrunner'},
                         'qs': 2},
 '6014925720352981095': {'chat_id': 327434945,
                         'message_id': 229,
                         'q': {'index': 0,
                               'link': 'https://www.bbc.co.uk/news/uk-54279802',
                               'sources': ['BBC, UK', 'Metro'],
                               'title': "Coronavirus: 'We're Aus

None
{'6014925720352981094': {'chat_id': 327434945,
                         'message_id': 225,
                         'q': {'index': 1,
                               'link': 'https://www.independent.co.uk/news/world/americas/us-politics/amy-coney-barrett-past-decisions-rulings-beliefs-supreme-court-trump-rbg-b556203.html',
                               'sources': ['The Sun', 'The Independent'],
                               'title': 'Amy Coney Barrett: The most '
                                        "controversial decisions by Trump's "
                                        'Supreme Court frontrunner'},
                         'qs': 2},
 '6014925720352981095': {'chat_id': 327434945,
                         'message_id': 229,
                         'q': {'index': 0,
                               'link': 'https://www.bbc.co.uk/news/uk-54279802',
                               'sources': ['BBC, UK', 'Metro'],
                               'title': "Coronavirus: 'We're Aus

None
{'6014925720352981094': {'chat_id': 327434945,
                         'message_id': 225,
                         'q': {'index': 1,
                               'link': 'https://www.independent.co.uk/news/world/americas/us-politics/amy-coney-barrett-past-decisions-rulings-beliefs-supreme-court-trump-rbg-b556203.html',
                               'sources': ['The Sun', 'The Independent'],
                               'title': 'Amy Coney Barrett: The most '
                                        "controversial decisions by Trump's "
                                        'Supreme Court frontrunner'},
                         'qs': 2},
 '6014925720352981095': {'chat_id': 327434945,
                         'message_id': 229,
                         'q': {'index': 0,
                               'link': 'https://www.bbc.co.uk/news/uk-54279802',
                               'sources': ['BBC, UK', 'Metro'],
                               'title': "Coronavirus: 'We're Aus

None
{'6014925720352981094': {'chat_id': 327434945,
                         'message_id': 225,
                         'q': {'index': 1,
                               'link': 'https://www.independent.co.uk/news/world/americas/us-politics/amy-coney-barrett-past-decisions-rulings-beliefs-supreme-court-trump-rbg-b556203.html',
                               'sources': ['The Sun', 'The Independent'],
                               'title': 'Amy Coney Barrett: The most '
                                        "controversial decisions by Trump's "
                                        'Supreme Court frontrunner'},
                         'qs': 2},
 '6014925720352981095': {'chat_id': 327434945,
                         'message_id': 229,
                         'q': {'index': 0,
                               'link': 'https://www.bbc.co.uk/news/uk-54279802',
                               'sources': ['BBC, UK', 'Metro'],
                               'title': "Coronavirus: 'We're Aus

None
{'6014925720352981094': {'chat_id': 327434945,
                         'message_id': 225,
                         'q': {'index': 1,
                               'link': 'https://www.independent.co.uk/news/world/americas/us-politics/amy-coney-barrett-past-decisions-rulings-beliefs-supreme-court-trump-rbg-b556203.html',
                               'sources': ['The Sun', 'The Independent'],
                               'title': 'Amy Coney Barrett: The most '
                                        "controversial decisions by Trump's "
                                        'Supreme Court frontrunner'},
                         'qs': 2},
 '6014925720352981095': {'chat_id': 327434945,
                         'message_id': 229,
                         'q': {'index': 0,
                               'link': 'https://www.bbc.co.uk/news/uk-54279802',
                               'sources': ['BBC, UK', 'Metro'],
                               'title': "Coronavirus: 'We're Aus

None
{'6014925720352981094': {'chat_id': 327434945,
                         'message_id': 225,
                         'q': {'index': 1,
                               'link': 'https://www.independent.co.uk/news/world/americas/us-politics/amy-coney-barrett-past-decisions-rulings-beliefs-supreme-court-trump-rbg-b556203.html',
                               'sources': ['The Sun', 'The Independent'],
                               'title': 'Amy Coney Barrett: The most '
                                        "controversial decisions by Trump's "
                                        'Supreme Court frontrunner'},
                         'qs': 2},
 '6014925720352981095': {'chat_id': 327434945,
                         'message_id': 229,
                         'q': {'index': 0,
                               'link': 'https://www.bbc.co.uk/news/uk-54279802',
                               'sources': ['BBC, UK', 'Metro'],
                               'title': "Coronavirus: 'We're Aus

None
{'6014925720352981094': {'chat_id': 327434945,
                         'message_id': 225,
                         'q': {'index': 1,
                               'link': 'https://www.independent.co.uk/news/world/americas/us-politics/amy-coney-barrett-past-decisions-rulings-beliefs-supreme-court-trump-rbg-b556203.html',
                               'sources': ['The Sun', 'The Independent'],
                               'title': 'Amy Coney Barrett: The most '
                                        "controversial decisions by Trump's "
                                        'Supreme Court frontrunner'},
                         'qs': 2},
 '6014925720352981095': {'chat_id': 327434945,
                         'message_id': 229,
                         'q': {'index': 0,
                               'link': 'https://www.bbc.co.uk/news/uk-54279802',
                               'sources': ['BBC, UK', 'Metro'],
                               'title': "Coronavirus: 'We're Aus

None
{'6014925720352981094': {'chat_id': 327434945,
                         'message_id': 225,
                         'q': {'index': 1,
                               'link': 'https://www.independent.co.uk/news/world/americas/us-politics/amy-coney-barrett-past-decisions-rulings-beliefs-supreme-court-trump-rbg-b556203.html',
                               'sources': ['The Sun', 'The Independent'],
                               'title': 'Amy Coney Barrett: The most '
                                        "controversial decisions by Trump's "
                                        'Supreme Court frontrunner'},
                         'qs': 2},
 '6014925720352981095': {'chat_id': 327434945,
                         'message_id': 229,
                         'q': {'index': 0,
                               'link': 'https://www.bbc.co.uk/news/uk-54279802',
                               'sources': ['BBC, UK', 'Metro'],
                               'title': "Coronavirus: 'We're Aus

None
{'6014925720352981094': {'chat_id': 327434945,
                         'message_id': 225,
                         'q': {'index': 1,
                               'link': 'https://www.independent.co.uk/news/world/americas/us-politics/amy-coney-barrett-past-decisions-rulings-beliefs-supreme-court-trump-rbg-b556203.html',
                               'sources': ['The Sun', 'The Independent'],
                               'title': 'Amy Coney Barrett: The most '
                                        "controversial decisions by Trump's "
                                        'Supreme Court frontrunner'},
                         'qs': 2},
 '6014925720352981095': {'chat_id': 327434945,
                         'message_id': 229,
                         'q': {'index': 0,
                               'link': 'https://www.bbc.co.uk/news/uk-54279802',
                               'sources': ['BBC, UK', 'Metro'],
                               'title': "Coronavirus: 'We're Aus

None
{'6014925720352981094': {'chat_id': 327434945,
                         'message_id': 225,
                         'q': {'index': 1,
                               'link': 'https://www.independent.co.uk/news/world/americas/us-politics/amy-coney-barrett-past-decisions-rulings-beliefs-supreme-court-trump-rbg-b556203.html',
                               'sources': ['The Sun', 'The Independent'],
                               'title': 'Amy Coney Barrett: The most '
                                        "controversial decisions by Trump's "
                                        'Supreme Court frontrunner'},
                         'qs': 2},
 '6014925720352981095': {'chat_id': 327434945,
                         'message_id': 229,
                         'q': {'index': 0,
                               'link': 'https://www.bbc.co.uk/news/uk-54279802',
                               'sources': ['BBC, UK', 'Metro'],
                               'title': "Coronavirus: 'We're Aus